In [ ]:
'''

Step 0
* X_o initial IOT
* X TARGET IOT
* u = X's row sums (KNOWN)
* v = X's column sums (KNOWN)
* additional non overlapping aggregation constraints w_{IJ}

Step 1
Define Z as X/X_o (elementwise)
* z = x/x_o where x_o <> 0
* z = 1 if x_o = 0


'''

In [5]:
import scipy as sp
import numpy as np

In [15]:
X = np.random.rand(10,10)-0.5
P = np.maximum(X,0)
N = np.maximum(-X,0)

assert np.all(X == P - N)

In [1]:
# given a matrix X, extract the positive part and the negative part such that X = P - N

In [ ]:
# define function p_i e p_j

# define function n_i e n_j

# define function p_IJ e n_IJ

# define s_j

# define r_j

# define t_IJ

In [ ]:
# step 0: define r_i and t_{IJ} = 1 for all IJ

In [2]:
# define tolerance 

# apply iteration procedure

```matlab
[m,n] = size(X0);
N = zeros(m,n);
N(X0<0) = -X0(X0<0); 
N = sparse(N);      %could save memory with large-scale matrices
P = X0+N;
P = sparse(P);
 
r = ones(m,1);      %initial guess for r (suggested by J&O, 2003)
pr = P'*r;
nr = N'*invd(r)*ones(m,1);
s1 = invd(2*pr)*(v+sqrt(v.^2+4*pr.*nr));    %first step s
ss = -invd(v)*nr;
s1(pr==0) = ss(pr==0);      
ps = P*s1;
ns = N*invd(s1)*ones(n,1);
r = invd(2*ps)*(u+sqrt(u.^2+4*ps.*ns));     %first step r
rr = - invd(u)*ns;
r(ps==0) = rr(ps==0);
 
pr = P'*r;
nr = N'*invd(r)*ones(m,1);
s2 = invd(2*pr)*(v+sqrt(v.^2+4*pr.*nr));    %second step s  
ss = -invd(v)*nr;
s2(pr==0) = ss(pr==0);  
dif = s2-s1;
iter = 1                %first iteration
if  nargin < 4 || isempty(eps)
    eps = 0.1e-5;       %default tolerance level
end        
M = max(abs(dif));
while (M > eps)
    s1 = s2;
    ps = P*s1;
    ns = N*invd(s1)*ones(n,1);
    r = invd(2*ps)*(u+sqrt(u.^2+4*ps.*ns));   %previous step r
    rr = -invd(u)*ns;
    r(ps==0) = rr(ps==0);
    pr = P'*r;
    nr = N'*invd(r)*ones(m,1);
    s2 = invd(2*pr)*(v+sqrt(v.^2+4*pr.*nr));  %current step s
    ss = -invd(v)*nr;
    s2(pr==0) = ss(pr==0);
    dif = s2-s1;
    iter = iter+1
    M = max(abs(dif));
end
s = s2;                                        %final step s
ps = P*s;
ns = N*invd(s)*ones(n,1);
r = invd(2*ps)*(u+sqrt(u.^2+4*ps.*ns));        %final step r
rr = -invd(u)*ns;
r(ps==0) = rr(ps==0);
X = diag(r)*P*diag(s)-invd(r)*N*invd(s);       %updated matrix
 
function invd = invd(x)         %auxiliary function used above
invd = 1./x;
invd(x==0) = 1;
invd = diag(invd);
```